In [55]:
import pygame
from screeninfo import get_monitors
import random
from random import randint
import math
import time

In [56]:
pygame.init()

(5, 0)

In [57]:
# Get Monitor Details of Primary Monitor

PrimaryMonitor = ""
for m in get_monitors():
    if(m.is_primary):
        PrimaryMonitor = m
        break

In [58]:
SCREEN_WIDTH = PrimaryMonitor.width
SCREEN_HEIGHT = PrimaryMonitor.height
# SCREEN_HEIGHT = PrimaryMonitor.height

print(SCREEN_WIDTH,SCREEN_HEIGHT)

1366 768


In [59]:
screen = pygame.display.set_mode((SCREEN_WIDTH - 10 , SCREEN_HEIGHT - 70))
DisplaySize = (screen.get_width(), screen.get_height())

In [60]:
backgroundImage = pygame.image.load('CrossRoad.png')
backgroundImage = pygame.transform.scale(backgroundImage, (SCREEN_WIDTH, SCREEN_HEIGHT))

In [61]:
font = pygame.font.Font('HighGameFont.ttf', 16)


In [62]:
blockSize = 15  # Set the size of the grid block
PositiveGridData = []
def drawGrid(background):
    gridData = []  # Initialize an empty list to store grid data

    for y in range(0, DisplaySize[1], blockSize):
        rowData = []  # Initialize an empty list for this row
        for x in range(0, DisplaySize[0], blockSize):
            rect = pygame.Rect(x, y, blockSize, blockSize)
            pygame.draw.rect(screen, (0, 0, 0), rect, 1)
            pixel_colors = [
                background.get_at((x + 1, y + 1)),
                background.get_at((x + blockSize - 1, y)),
                background.get_at((x, y + blockSize - 1)),
                background.get_at((x + blockSize - 1, y + blockSize - 1)),
                background.get_at((x + blockSize // 2, y + blockSize // 2))
            ]

            if ((1, 165, 79, 255) in pixel_colors) or ((255, 255, 255, 255)) in pixel_colors:
                text = font.render("0", True, (0, 0, 0))
                cellValue = 0  # Mark this cell as "0"
            else:
                text = font.render("1", True, (0, 0, 0))
                cellValue = 1  # Mark this cell as "1"
            
            textRect = text.get_rect()
            textRect.center = (x + blockSize // 2, y + blockSize // 2)
            screen.blit(text, textRect)

            rowData.append((x, y, x+blockSize, y+blockSize ,cellValue))  # Add cell value to row data
        gridData.append(rowData)  # Add row data to grid data
    print("Start")
    print(gridData)
    print("End")
    return gridData  # Return the generated grid data for further use or saving


In [63]:
class Character:
    def __init__(self, currentX, currentY, endX, endY, speed, color, characterLength):
        self.currentX = currentX
        self.currentY = currentY
        self.endX = endX
        self.endY = endY
        self.speed = speed
        self.color = color
        self.characterLength = characterLength
        self.current_position = (currentX, currentY)
        self.is_moving = False
        self.isCrashed = False
        self.inIntersection = False
        self.intersectionCounts = 0


    
    def move(self, OnIntersection):
        if (abs(self.endX - self.currentX) <= (self.characterLength)) and (abs(self.endY - self.currentY) <= 2):
            self.is_moving = False
            return
        else:
            self.is_moving = True

        self.inIntersection = OnIntersection
        self.intersectionCounts += 1 if(OnIntersection) else 0
        
        if(self.inIntersection or self.intersectionCounts > 0):
            if abs(self.currentX - self.endX) <= 5:
                self.currentX = self.endX
            else:
                if self.currentX < self.endX:
                    self.currentX += self.speed
                elif self.currentX > self.endX:
                    self.currentX -= self.speed

            if abs(self.currentY - self.endY) <= 5:
                self.currentY = self.endY
            else:
                if self.currentY < self.endY:
                    self.currentY += self.speed
                elif self.currentY > self.endY:
                    self.currentY -= self.speed
        else:
            if (self.currentX < SCREEN_WIDTH*0.33 or self.currentX > SCREEN_WIDTH*0.65):
                if abs(self.currentX - self.endX) <= 5:
                    self.currentX = self.endX
                else:
                    if self.currentX < self.endX:
                        self.currentX += self.speed
                    elif self.currentX > self.endX:
                        self.currentX -= self.speed
            elif(self.currentY < SCREEN_HEIGHT*0.20 or self.currentY > SCREEN_HEIGHT*0.65):
                if abs(self.currentY - self.endY) <= 5:
                    self.currentY = self.endY
                else:
                    if self.currentY < self.endY:
                        self.currentY += self.speed
                    elif self.currentY > self.endY:
                        self.currentY -= self.speed
            
        self.current_position = (self.currentX, self.currentY)

    def get_current_position(self):
        return self.current_position
    
    def set_current_position(self, currentX, currentY):
        self.currentX = currentX
        self.currentY = currentY
        self.current_position = (currentX, currentY)


In [64]:
PossibleLanes = {
                    'TopSide': {
                        "Lane1": (SCREEN_WIDTH*0.36, -50),
                        "Lane2": (SCREEN_WIDTH*0.504, -50),
                        "Lane3": (SCREEN_WIDTH*0.656, -50)
                    },
                    'RightSide': {
                        "Lane4": (SCREEN_WIDTH+50, SCREEN_HEIGHT*0.248),
                        "Lane5": (SCREEN_WIDTH+50, SCREEN_HEIGHT*0.411),
                        "Lane6": (SCREEN_WIDTH+50, SCREEN_HEIGHT*0.577)
                    },
                    'BottomSide': {
                        "Lane7": (SCREEN_WIDTH*0.655, SCREEN_HEIGHT+50),
                        "Lane8": (SCREEN_WIDTH*0.503, SCREEN_HEIGHT+50),
                        "Lane9": (SCREEN_WIDTH*0.36, SCREEN_HEIGHT+50),
                    },
                    'LeftSide': {
                        "Lane10": (-50, SCREEN_HEIGHT*0.576),
                        "Lane11": (-50, SCREEN_HEIGHT*0.41),
                        "Lane12": (-50, SCREEN_HEIGHT*0.247),
                    }
                }

Vehicles = []
count = 0
while count<=1:
    StartSide = random.choice(list(PossibleLanes.items()))
    EndSide = random.choice(list(PossibleLanes.items()))
    if(StartSide[0] == EndSide[0]):
        count -= 1
        continue
    Start = random.choice(list(StartSide[1].items()))
    End = random.choice(list(EndSide[1].items()))

    Vehicles.append(Character(Start[1][0], Start[1][1], End[1][0], End[1][1], ((randint(50, 100)//10)),(randint(0, 255),randint(0, 255),randint(0, 255)), (randint(10, 20))))
    count += 1

In [65]:
def check_collision(circle1, circle2):
    x1, y1, r1 = circle1.currentX, circle1.currentY, circle1.characterLength
    x2, y2, r2 = circle2.currentX, circle2.currentY, circle2.characterLength

    # Calculate the distance between the centers of the circles
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

    # Check if the distance is less than or equal to the sum of the radii
    if distance <= r1 + r2:
        return True  # Collision detected
    else:
        return False  # No collision

In [66]:
TopLeft = (SCREEN_WIDTH*0.31, 0)
TopRight = (SCREEN_WIDTH*0.717, 0)

MidTopLeft = (0, SCREEN_HEIGHT*0.18)
MidTopRight = (SCREEN_WIDTH*0.992, SCREEN_HEIGHT*0.1859)

MidBottomLeft = (0, SCREEN_HEIGHT*0.68)
MidBottomRight = (SCREEN_WIDTH*0.992, SCREEN_HEIGHT*0.685)

BottomLeft = (SCREEN_WIDTH*0.31, SCREEN_HEIGHT*0.931)
BottomRight = (SCREEN_WIDTH*0.717, SCREEN_HEIGHT*0.931)

CenterTopLeft = (SCREEN_WIDTH*0.31, SCREEN_HEIGHT*0.18)
CenterTopRight = (SCREEN_WIDTH*0.717, SCREEN_HEIGHT*0.18)

CenterBottomLeft = (SCREEN_WIDTH*0.31, SCREEN_HEIGHT*0.69)
CenterBottomRight = (SCREEN_WIDTH*0.717, SCREEN_HEIGHT*0.69)


PixelDimentions = (SCREEN_WIDTH*0.002, SCREEN_HEIGHT*0.0033)
PixelColor = (255, 0, 0)

In [67]:
def distance(p1, p2):
    return math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)

def circle_line_intersection(circle_center, radius, line_start, line_end):
    # Calculate the distance between the line start and end points
    line_length = distance(line_start, line_end)

    # Calculate the direction vector of the line
    direction = ((line_end[0] - line_start[0]) / line_length, (line_end[1] - line_start[1]) / line_length)

    # Calculate the vector from the line start to the circle center
    circle_vector = (circle_center[0] - line_start[0], circle_center[1] - line_start[1])

    # Calculate the projection of the circle vector onto the line direction
    projection = circle_vector[0] * direction[0] + circle_vector[1] * direction[1]

    # Calculate the closest point on the line to the circle center
    closest_point = (line_start[0] + projection * direction[0], line_start[1] + projection * direction[1])

    # Calculate the distance between the closest point and the circle center
    distance_to_line = distance(circle_center, closest_point)

    # Check if the distance to the line is less than or equal to the circle radius
    if distance_to_line <= radius:
        return True  # Circle intersects with the line
    # else:
    #     return False  # Circle does not intersect with the line

In [68]:
def is_circle_in_rectangle(rect_x1, rect_y1, rect_x2, rect_y2, circle_x, circle_y, radius):
    # Check if the circle is within the rectangle bounds
    if (circle_x - radius >= rect_x1 and
        circle_x + radius <= rect_x2 and
        circle_y - radius >= rect_y1 and
        circle_y + radius <= rect_y2):
        return True  # Circle is entirely within the rectangle
    else:
        return False  # Circle is not entirely within the rectangle

In [69]:
def is_outside_display(width, height, x, y):
    # Check if the coordinate is outside the display bounds
    if x < 0 or x >= width or y < 0 or y >= height:
        return True  # The coordinate is outside the display
    else:
        return False  # The coordinate is inside the display

In [70]:
def check_neighbors(data, row, col):
    max_rows = len(data)
    max_cols = len(data[0])  # Assuming all rows have equal length

    # Initialize neighbors to None
    top, bottom, left, right = (0,0,0,0,0), (0,0,0,0,0), (0,0,0,0,0), (0,0,0,0,0)

    # Check top neighbor
    if row > 0:
        top = data[row - 1][col]

    # Check bottom neighbor
    if row < max_rows - 1:
        bottom = data[row + 1][col]

    # Check left neighbor
    if col > 0:
        left = data[row][col - 1]

    # Check right neighbor
    if col < max_cols - 1:
        right = data[row][col + 1]

    return top, bottom, left, right

In [71]:
collitions = 0
GridData = drawGrid(backgroundImage)
GridData2 = []
for row_idx, row in enumerate(GridData):
    for col_idx, element in enumerate(row):
        start_x, start_y, end_x, end_y, value = element
        top, bottom, left, right = check_neighbors(GridData, row_idx, col_idx)
        if(value and (top[4] and bottom[4] and left[4] and right[4])):
            pygame.draw.rect(screen, (0,0,0), pygame.Rect((start_x, start_y), (blockSize,blockSize)), 1)
            GridData2.append((start_x, start_y, end_x, end_y, 1))


run = True
while run:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            # Get the mouse position and print it
            mouse_x, mouse_y = pygame.mouse.get_pos()
            print(f"Mouse clicked at ({mouse_x}, {mouse_y})")

    keys = pygame.key.get_pressed()

    screen.blit(backgroundImage, (0, 0))

    # for i in GridData2:
    #     pygame.draw.rect(screen, (0,0,0), pygame.Rect((i[0], i[1]), (blockSize,blockSize)), 1)


    for character in Vehicles:
        character.move(is_circle_in_rectangle(*CenterTopLeft, *CenterBottomRight, character.currentX, character.currentY, character.characterLength))
        pygame.draw.circle(screen, character.color,[*character.current_position], character.characterLength)


        text = font.render('Total Collition: ' + str(collitions), True, (255,255,255))
        textRect = text.get_rect()
        textRect.center = (150, 50)
        screen.blit(text, textRect)

        for character2 in Vehicles:
            if ((character != character2) and (not character.isCrashed) and (not character2.isCrashed)):
                if check_collision(character, character2):
                    collitions += 1

                    character.speed = 0
                    character.isCrashed = True

                    character2.speed = 0
                    character2.isCrashed = True

                    if(character in Vehicles):
                        Vehicles.remove(character)
                    if(character2 in Vehicles):
                        Vehicles.remove(character2)
                    for i in range(2):
                        StartSide = random.choice(list(PossibleLanes.items()))
                        EndSide = random.choice(list(PossibleLanes.items()))
                        if(StartSide[0] == EndSide[0]):
                            count -= 1
                            continue
                        Start = random.choice(list(StartSide[1].items()))
                        End = random.choice(list(EndSide[1].items()))

                        Vehicles.append(Character(Start[1][0], Start[1][1], End[1][0], End[1][1], ((randint(50, 100)//10)),(randint(0, 255),randint(0, 255),randint(0, 255)), (randint(10, 20))))



        if(is_outside_display(DisplaySize[0], DisplaySize[1], character.currentX, character.currentY) and character.intersectionCounts>0):
            if(character in Vehicles):
                Vehicles.remove(character)
            for i in range(2):
                StartSide = random.choice(list(PossibleLanes.items()))
                EndSide = random.choice(list(PossibleLanes.items()))
                if(StartSide[0] == EndSide[0]):
                    count -= 1
                    continue
                Start = random.choice(list(StartSide[1].items()))
                End = random.choice(list(EndSide[1].items()))

                Vehicles.append(Character(Start[1][0], Start[1][1], End[1][0], End[1][1], ((randint(50, 100)//10)),(randint(0, 255),randint(0, 255),randint(0, 255)), (randint(10, 20))))










    # Pixel Border Markers:
    # pygame.draw.rect(screen, PixelColor, pygame.Rect((TopLeft), PixelDimentions))
    # pygame.draw.rect(screen, PixelColor, pygame.Rect((TopRight), PixelDimentions))
    # pygame.draw.rect(screen, PixelColor, pygame.Rect((BottomLeft), PixelDimentions))
    # pygame.draw.rect(screen, PixelColor, pygame.Rect((BottomRight), PixelDimentions))
    # pygame.draw.rect(screen, PixelColor, pygame.Rect((MidTopLeft), PixelDimentions))
    # pygame.draw.rect(screen, PixelColor, pygame.Rect((MidTopRight), PixelDimentions))
    # pygame.draw.rect(screen, PixelColor, pygame.Rect((MidBottomLeft), PixelDimentions))
    # pygame.draw.rect(screen, PixelColor, pygame.Rect((MidBottomRight), PixelDimentions))
    # pygame.draw.rect(screen, PixelColor, pygame.Rect((CenterTopLeft), PixelDimentions))
    # pygame.draw.rect(screen, PixelColor, pygame.Rect((CenterTopRight), PixelDimentions))
    # pygame.draw.rect(screen, PixelColor, pygame.Rect((CenterBottomLeft), PixelDimentions))
    # pygame.draw.rect(screen, PixelColor, pygame.Rect((CenterBottomRight), PixelDimentions))
                

                
    # Pixel Border Line Markers:
    pygame.draw.line(screen, PixelColor, (TopLeft), (TopRight), 5)
    pygame.draw.line(screen, PixelColor, (BottomLeft), (BottomRight), 5)
    pygame.draw.line(screen, PixelColor, (MidTopLeft), (MidBottomLeft), 5)
    pygame.draw.line(screen, PixelColor, (MidTopRight), (MidBottomRight), 5)
    pygame.draw.line(screen, PixelColor, (CenterTopLeft), (CenterTopRight), 5)
    pygame.draw.line(screen, PixelColor, (CenterBottomLeft), (CenterBottomRight), 5)
    pygame.draw.line(screen, PixelColor, (CenterTopLeft), (CenterBottomLeft), 5)
    pygame.draw.line(screen, PixelColor, (CenterTopRight), (CenterBottomRight), 5)

    

    # Update the display
    pygame.display.flip()
        
    

Start
[[(0, 0, 15, 15, 0), (15, 0, 30, 15, 0), (30, 0, 45, 15, 0), (45, 0, 60, 15, 0), (60, 0, 75, 15, 0), (75, 0, 90, 15, 0), (90, 0, 105, 15, 0), (105, 0, 120, 15, 0), (120, 0, 135, 15, 0), (135, 0, 150, 15, 0), (150, 0, 165, 15, 0), (165, 0, 180, 15, 0), (180, 0, 195, 15, 0), (195, 0, 210, 15, 0), (210, 0, 225, 15, 0), (225, 0, 240, 15, 0), (240, 0, 255, 15, 0), (255, 0, 270, 15, 0), (270, 0, 285, 15, 0), (285, 0, 300, 15, 0), (300, 0, 315, 15, 0), (315, 0, 330, 15, 0), (330, 0, 345, 15, 0), (345, 0, 360, 15, 0), (360, 0, 375, 15, 0), (375, 0, 390, 15, 0), (390, 0, 405, 15, 0), (405, 0, 420, 15, 0), (420, 0, 435, 15, 0), (435, 0, 450, 15, 1), (450, 0, 465, 15, 1), (465, 0, 480, 15, 1), (480, 0, 495, 15, 1), (495, 0, 510, 15, 1), (510, 0, 525, 15, 1), (525, 0, 540, 15, 1), (540, 0, 555, 15, 1), (555, 0, 570, 15, 1), (570, 0, 585, 15, 1), (585, 0, 600, 15, 1), (600, 0, 615, 15, 1), (615, 0, 630, 15, 1), (630, 0, 645, 15, 1), (645, 0, 660, 15, 1), (660, 0, 675, 15, 1), (675, 0, 690, 15

In [72]:
pygame.quit()